<a href="https://colab.research.google.com/github/Jozdien/reward-side-channels/blob/main/Reward_Side_Channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 0s (6,396 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...

In [4]:

# Special gym environment
!pip install gym[atari]

# For rendering environment, you can use pyvirtualdisplay.
!pip install pyvirtualdisplay
!pip install piglet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 16.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 4.7 MB/s 


In [5]:
#Install spinningup on CoLab
!git clone https://github.com/AmritaANair/spinningup-logchanges.git
!cd spinningup-logchanges
#!pip install -e . # this will incur error: File "setup.py" not found. Directory cannot be installed in editable mode: /content
!pip install -e spinningup-logchanges

Cloning into 'spinningup-logchanges'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 243 (delta 41), reused 225 (delta 29), pack-reused 0
Receiving objects: 100% (243/243), 23.78 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/spinningup-logchanges
     |████████████████████████████████| 1.6 MB 18.5 MB/s 
     |████████████████████████████████| 13.1 MB 48.4 MB/s 
     |████████████████████████████████| 2.5 MB 50.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 178 kB 65.9 MB/s 
     |████████████████████████████████| 110.5 MB 1.3 kB/s 
     |████████████████████████████████| 734.6 MB 16 kB/s 
     |█████████████████████

## **Note:** After Running the previous (4) blocks of code, restart the runtime. Once that's done, just run the code from this point on (need not run the first 4 blocks of installation code again!)

In [1]:

# To activate virtual display 
# need to run a script once for training an agent as follows
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    %%bash ../xvfb start
    %env DISPLAY=:1

#
# Import libraries
#
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [2]:
from spinup import ppo_pytorch as ppo
import torch
import gym
import os.path as osp, time, atexit, os
import json, sys, os, datetime

env_fn = lambda : gym.make('LunarLander-v2')

ac_kwargs = dict(hidden_sizes=[64,64], activation=torch.nn.ReLU)

logger_kwargs = dict(output_dir='gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output', exp_name='experiment_name')

In [3]:
ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=500, epochs=20, logger_kwargs=logger_kwargs)  #changed from steps_per_epoch=5000, epochs=250

Logging data to gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"ReLU",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.2,
    "env_fn":	"<function <lambda> at 0x7f9a0c176d40>",
    "epochs":	20,
    "exp_name":	"experiment_name",
    "gamma":	0.99,
    "lam":	0.97,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7f99af93dbd0>":	{
            "epoch_dict":	{},
            "exp_name":	"experiment_name",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output",
            "output_file":	{
                "<_io.TextIOWrapper name='gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"


In [4]:
import shutil

original = r'gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output'
target = r'gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/ppo-agent-results-ep-20-'+ datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

shutil.copytree(original, target)

'gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/ppo-agent-results-ep-20-2022-09-21_08-59-06'

In [5]:
from spinup.utils.test_policy import load_policy_and_env as load_policy
# Show policy
from spinup.utils.test_policy import run_policy
_, get_action = load_policy('gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output')
env2 = gym.make('LunarLander-v2')
env3 = wrap_env(env2)
run_policy(env3, get_action, max_ep_len=1000, num_episodes=10)
env3.close()
show_video() 



Loading from gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output/pyt_save/model.pt.


Logging data to /content/gdrive/MyDrive/Reward Side Channels - Model Data/tmp/experiments/ppo-agent-results-ep-20-2022-09-21_08-59-07/progress.txt
Episode 0 	 EpRet -414.078 	 EpLen 361
Episode 1 	 EpRet -160.282 	 EpLen 121
Episode 2 	 EpRet -135.054 	 EpLen 1000
Episode 3 	 EpRet -15.039 	 EpLen 126
Episode 4 	 EpRet -48.351 	 EpLen 241
Episode 5 	 EpRet -37.936 	 EpLen 130
Episode 6 	 EpRet -145.525 	 EpLen 394
Episode 7 	 EpRet -87.245 	 EpLen 215
Episode 8 	 EpRet -100.385 	 EpLen 233
Episode 9 	 EpRet 5.713 	 EpLen 275
-------------------------------------
|    AverageEpRet |            -114 |
|        StdEpRet |             113 |
|        MaxEpRet |            5.71 |
|        MinEpRet |            -414 |
|           EpLen |             310 |
-------------------------------------
